In [1]:
import datetime
from tkinter import *
import tkinter.messagebox as mb
from tkinter import ttk
from tkcalendar import DateEntry
from sqlalchemy import create_engine, Column, String, Integer, Date
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

In [2]:
# Creating the universal font variables
headlabelfont = ("Noto Sans CJK TC", 15, 'bold')
labelfont = ('Garamond', 14)
entryfont = ('Garamond', 12)

# Creating the SQLAlchemy engine and session
engine = create_engine('sqlite:///SchoolManagement.db', echo=True)
Base = declarative_base()
Session = sessionmaker(bind=engine)
session = Session()

In [3]:
# Creating the Student model
class Student(Base):
    __tablename__ = 'students'
    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String)
    email = Column(String)
    phone_no = Column(String)
    gender = Column(String)
    dob = Column(Date)
    stream = Column(String)


Base.metadata.create_all(engine)

2023-07-03 21:28:58,075 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-07-03 21:28:58,077 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("students")
2023-07-03 21:28:58,077 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-07-03 21:28:58,077 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("students")
2023-07-03 21:28:58,083 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-07-03 21:28:58,083 INFO sqlalchemy.engine.Engine 
CREATE TABLE students (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	email VARCHAR, 
	phone_no VARCHAR, 
	gender VARCHAR, 
	dob DATE, 
	stream VARCHAR, 
	PRIMARY KEY (id)
)


2023-07-03 21:28:58,086 INFO sqlalchemy.engine.Engine [no key 0.00146s] ()
2023-07-03 21:28:58,095 INFO sqlalchemy.engine.Engine COMMIT


In [4]:
# Creating the functions
def reset_fields():
    global name_strvar, email_strvar, contact_strvar, gender_strvar, dob, stream_strvar

    for var in [name_strvar, email_strvar, contact_strvar, gender_strvar, stream_strvar]:
        var.set('')
    dob.set_date(datetime.datetime.now().date())


def reset_form():
    global tree
    tree.delete(*tree.get_children())

    reset_fields()

In [5]:
def display_records():
    tree.delete(*tree.get_children())

    records = session.query(Student).all()

    for record in records:
        tree.insert('', END, values=(
            record.id, record.name, record.email, record.phone_no, record.gender, record.dob, record.stream))


def add_record():
    global name_strvar, email_strvar, contact_strvar, gender_strvar, dob, stream_strvar

    name = name_strvar.get()
    email = email_strvar.get()
    contact = contact_strvar.get()
    gender = gender_strvar.get()
    dob_val = dob.get_date()
    stream = stream_strvar.get()

    if not name or not email or not contact or not gender or not dob_val or not stream:
        mb.showerror('Error!', "Please fill all the missing fields!!")
    else:
        try:
            student = Student(name=name, email=email, phone_no=contact, gender=gender, dob=dob_val, stream=stream)
            session.add(student)
            session.commit()
            mb.showinfo('Record added', f"Record of {name} was successfully added")
            reset_fields()
            display_records()
        except Exception as e:
            mb.showerror('Error!', str(e))

In [6]:
def remove_record():
    if not tree.selection():
        mb.showerror('Error!', 'Please select an item from the database')
    else:
        current_item = tree.focus()
        values = tree.item(current_item)
        selection = values["values"]

        tree.delete(current_item)

        student = session.query(Student).get(selection[0])
        if student:
            session.delete(student)
            session.commit()
            mb.showinfo('Done', 'The record you wanted deleted was successfully deleted.')
        else:
            mb.showerror('Error!', 'Failed to delete the record.')

        display_records()

In [7]:
def view_record():
    global name_strvar, email_strvar, contact_strvar, gender_strvar, dob, stream_strvar

    current_item = tree.focus()
    values = tree.item(current_item)
    selection = values["values"]

    name_strvar.set(selection[1])
    email_strvar.set(selection[2])
    contact_strvar.set(selection[3])
    gender_strvar.set(selection[4])
    dob.set_date(selection[5])
    stream_strvar.set(selection[6])

In [8]:
def update_record():
    if not tree.selection():
        mb.showerror('Error!', 'Please select an item from the database')
    else:
        global name_strvar, email_strvar, contact_strvar, gender_strvar, dob, stream_strvar
        current_item = tree.focus()
        values = tree.item(current_item)
        selection = values["values"]

        name = name_strvar.get()
        email = email_strvar.get()
        contact = contact_strvar.get()
        gender = gender_strvar.get()
        dob_val = dob.get_date()
        stream = stream_strvar.get()

        if not name or not email or not contact or not gender or not dob_val or not stream:
            mb.showerror('Error!', "Please fill all the missing fields!!")
        else:
            try:
                student = Student(id=selection[0], name=name, email=email, phone_no=contact, gender=gender,
                                  dob=dob_val, stream=stream)
                session.merge(student)
                session.commit()
                mb.showinfo('Record updated', f"Record of {name} was successfully updated")
                reset_fields()
                display_records()
            except Exception as e:
                mb.showerror('Error!', str(e))

In [9]:
# Initializing the GUI window
main = Tk()
main.title('School Management System')
main.geometry('1250x660')
main.resizable(0, 0)

''

In [10]:
# Creating the background and foreground color variables
lf_bg = 'MediumSpringGreen'  # bg color for the left_frame
cf_bg = 'PaleGreen'  # bg color for the center_frame

In [11]:
# Creating the StringVar or IntVar variables
name_strvar = StringVar()
email_strvar = StringVar()
contact_strvar = StringVar()
gender_strvar = StringVar()
stream_strvar = StringVar()

In [12]:
# Placing the components in the main window
Label(main, text="SCHOOL MANAGEMENT SYSTEM", font=headlabelfont, bg='SpringGreen').pack(side=TOP, fill=X)

left_frame = Frame(main, bg=lf_bg)
left_frame.place(x=0, y=30, relheight=1, relwidth=0.2)

center_frame = Frame(main, bg=cf_bg)
center_frame.place(relx=0.2, y=30, relheight=1, relwidth=0.2)

right_frame = Frame(main, bg="Gray35")
right_frame.place(relx=0.4, y=30, relheight=1, relwidth=0.6)

In [13]:
# Placing components in the left frame
Label(left_frame, text="Full Name", font=headlabelfont, bg=lf_bg).place(relx=0.3, rely=0.05)
Label(left_frame, text="Contact Number", font=headlabelfont, bg=lf_bg).place(relx=0.175, rely=0.18)
Label(left_frame, text="Email Address", font=headlabelfont, bg=lf_bg).place(relx=0.2, rely=0.31)
Label(left_frame, text="Gender", font=headlabelfont, bg=lf_bg).place(relx=0.3, rely=0.44)
Label(left_frame, text="Date of Birth", font=headlabelfont, bg=lf_bg).place(relx=0.2, rely=0.57)
Label(left_frame, text="Stream", font=headlabelfont, bg=lf_bg).place(relx=0.3, rely=0.7)

Entry(left_frame, width=19, textvariable=name_strvar, font=entryfont).place(x=20, rely=0.1)
Entry(left_frame, width=19, textvariable=contact_strvar, font=entryfont).place(x=20, rely=0.23)
Entry(left_frame, width=19, textvariable=email_strvar, font=entryfont).place(x=20, rely=0.36)
Entry(left_frame, width=19, textvariable=stream_strvar, font=entryfont).place(x=20, rely=0.75)

OptionMenu(left_frame, gender_strvar, 'Male', 'Female').place(x=45, rely=0.49, relwidth=0.5)

dob = DateEntry(left_frame, font=('Arial', 12), width=15)
dob.place(x=20, rely=0.62)

Button(left_frame, text='Submit', font=headlabelfont, command=add_record, width=18, bg='#262D37', fg='LightCyan').place(relx=0.025, rely=0.85)


In [14]:
# Placing components in the center frame
Button(center_frame, text='Delete Record', font=headlabelfont, command=remove_record, width=15, bg='black', fg='LightCyan').place(relx=0.1, rely=0.25)
Button(center_frame, text='View Record', font=headlabelfont, command=view_record, width=15, bg='black', fg='LightCyan').place(relx=0.1, rely=0.35)
Button(center_frame, text='Update', font=headlabelfont, command=update_record, width=15, bg='black', fg='LightCyan').place(relx=0.1, rely=0.45)
Button(center_frame, text='Reset Fields', font=headlabelfont, command=reset_fields, width=15, bg='black', fg='LightCyan').place(relx=0.1, rely=0.55)
Button(center_frame, text='Delete database', font=headlabelfont, command=reset_form, width=15, bg='Red', fg='LightCyan').place(relx=0.1, rely=0.65)

In [15]:
# Placing components in the right frame
Label(right_frame, text='Students Records', font=headlabelfont, bg='DarkGreen', fg='LightCyan').pack(side=TOP, fill=X)

tree = ttk.Treeview(right_frame, height=100, selectmode=BROWSE,
                    columns=('Student ID', 'Name', 'Email Address', 'Contact Number', 'Gender', 'Date of Birth', 'Stream'))

X_scroller = Scrollbar(tree, orient=HORIZONTAL, command=tree.xview)
Y_scroller = Scrollbar(tree, orient=VERTICAL, command=tree.yview)

X_scroller.pack(side=BOTTOM, fill=X)
Y_scroller.pack(side=RIGHT, fill=Y)

tree.config(yscrollcommand=Y_scroller.set, xscrollcommand=X_scroller.set)

tree.heading('Student ID', text='ID', anchor=CENTER)
tree.heading('Name', text='Name', anchor=CENTER)
tree.heading('Email Address', text='Email ID', anchor=CENTER)
tree.heading('Contact Number', text='Phone No', anchor=CENTER)
tree.heading('Gender', text='Gender', anchor=CENTER)
tree.heading('Date of Birth', text='DOB', anchor=CENTER)
tree.heading('Stream', text='Stream', anchor=CENTER)

tree.column('#0', width=0, stretch=NO)
tree.column('#1', width=50, anchor=CENTER)
tree.column('#2', width=150, anchor=CENTER)
tree.column('#3', width=150, anchor=CENTER)
tree.column('#4', width=150, anchor=CENTER)
tree.column('#5', width=100, anchor=CENTER)
tree.column('#6', width=100, anchor=CENTER)

tree.pack(side=TOP, fill=BOTH, expand=True)

display_records()

main.mainloop()

2023-07-03 21:33:03,538 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-07-03 21:33:03,538 INFO sqlalchemy.engine.Engine SELECT students.id AS students_id, students.name AS students_name, students.email AS students_email, students.phone_no AS students_phone_no, students.gender AS students_gender, students.dob AS students_dob, students.stream AS students_stream 
FROM students
2023-07-03 21:33:03,538 INFO sqlalchemy.engine.Engine [generated in 0.00119s] ()
2023-07-03 21:34:01,491 INFO sqlalchemy.engine.Engine INSERT INTO students (name, email, phone_no, gender, dob, stream) VALUES (?, ?, ?, ?, ?, ?)
2023-07-03 21:34:01,491 INFO sqlalchemy.engine.Engine [generated in 0.00204s] ('Rahul', 'ggu@xyz.com', '9898777888', 'Male', '2011-05-10', 'Maths')
2023-07-03 21:34:01,498 INFO sqlalchemy.engine.Engine COMMIT
2023-07-03 21:34:03,573 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-07-03 21:34:03,575 INFO sqlalchemy.engine.Engine SELECT students.id AS students_id, students.name AS stude

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\kotar\anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\kotar\AppData\Local\Temp\ipykernel_1952\2128377924.py", line 8, in view_record
    name_strvar.set(selection[1])
IndexError: string index out of range
